In [1]:
import fasttext
from huggingface_hub import hf_hub_download

model_path = hf_hub_download(repo_id="facebook/fasttext-language-identification", filename="model.bin")
language_identification = fasttext.load_model(model_path)


/opt/miniconda3/envs/crawl_kinyarwanda/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pandas as pd
df = pd.read_csv("~/Downloads/cc_websites.csv.gz", compression="gzip")

In [6]:
urls = []
for language in df.content_languages.unique():
    urls.append(df.query("content_languages==@language").sample(n=30, random_state=0))
urls = pd.concat(urls)

for ix, row in urls.iterrows():
    print(row["content_languages"], row["url"])

kin https://www.alenamaze.com/forum
kin http://kanda.ayz.pl/index.php/6aj98_v33xd42e8e72zd-g_t8rzrlp
kin http://kanda.ayz.pl/index.php/0cevjo580ll27784541awuec5aikd
kin https://www.tebyan.net/bigimage.aspx?img=http://img.tebyan.net/big/1391/04/20713267121114192126131231672011930187132175.jpg
kin http://mail.igihe.com/imyemerere/article/uko-umuhango-wo-kwimika-musenyeri-musengamana-papias-wa-diyosezi-ya-byumba
kin https://www.kigalitoday.com/amakuru/mu-mahanga/article/tanzania-amatangazo-yamamaza-anyura-kuri-interineti-agiye-kujya-asoreshwa
kin https://igihe.com/forum_iframe.php?id_article=84526&start=0
kin https://igicumbinews.co.rw/category/imikino/
kin https://gitweb.gentoo.org/data/glsa.git/diff/?id=f62be9710b7244af8dc15887df52ae442c460d45
kin https://www.ikigega.rw/index.php?id=10&tx_news_pi1%5Bnews%5D=88&tx_news_pi1%5Bcontroller%5D=News&tx_news_pi1%5Baction%5D=detail&cHash=098175a07d9e0718dafa5abf18c801ee
kin https://umuseke.rw/category/izamamaza/amp/
kin https://www.kigalitoday.c

In [7]:
import sys
sys.path.append("../../crawler")
from crawler import download, CrawlerConfig, HTML2Text

In [8]:
from tqdm.contrib.concurrent import process_map

def download_urls(urls):
    config = CrawlerConfig()
    batch = [(url, config) for url in urls]
    html = process_map(download, batch, max_workers=8)
    return html

us = ["https://goobjoog.com/2024/10/08/baarlamaanka-dalka-kenya-oo-xilka-ka-qaaday-madaxweyne-kuxigeenka-dalkaasi-rigathi-gachagua/", "https://so.ihorror.com/carada-becky-ayaa-sii-socota-beckys-gore-buuxiyey-sheeko-aargoosi-ah/"]
pages = download_urls(urls.url)

100%|██████████| 480/480 [03:16<00:00,  2.44it/s]


In [9]:
from bs4 import BeautifulSoup
import json
import numpy as np

html2text = HTML2Text()

def process_page(page):
    if not page[1]:
        return None
    
    page = json.loads(page[0])
    soup = BeautifulSoup(page["html"], "html.parser")
    texts = html2text.extract_text(soup)
    if len(texts) == 0:
        return None
    langs = [language_identification.predict(t)[0][0][len("__label__"):] for t in texts]

    lang, count = np.unique(langs, return_counts=True)
    count = count / count.sum()
    lang = {lang[i]: count[i] for i in range(len(lang))}
    max_lang = max(lang, key=lang.get)

    if lang[max_lang] > 0.6:
        return max_lang
    else:
        return None

langs = [process_page(page) for page in pages]
urls["language"] = langs
urls

,url,content_languages,language
3401538,https://www.alenamaze.com/forum,kin,eng_Latn
4056286,http://kanda.ayz.pl/index.php/6aj98_v33xd42e8e...,kin,None
4416697,http://kanda.ayz.pl/index.php/0cevjo580ll27784...,kin,None
2928356,https://www.tebyan.net/bigimage.aspx?img=http:...,kin,None
3982248,http://mail.igihe.com/imyemerere/article/uko-u...,kin,None
...,...,...,...
136075,https://embed.vietnamnettv.vn/v/231165.html,aka,None
272168,https://mssg.me/biscuit_abakan,aka,None
2237198,https://embed.vietnamnettv.vn/v/227280.html,aka,None
1178195,http://www.ntra.com/ntra-member-profile/?ntrai...,aka,fra_Latn


In [11]:
urls = urls.query("language.notna()").copy()
urls["detected_language"] = urls.language.apply(lambda x:x[0:3])
urls = urls.query("content_languages==detected_language")

languages = {}
for lang in urls.content_languages.unique():
    languages[lang] = urls.query("content_languages==@lang").language.unique().tolist()[0]

df = pd.read_csv("data/languages.csv")

for lang in df.code.unique():
    if lang not in languages.keys():
        languages[lang] = lang + "_Latn"
        print(f"set {lang} to Latn script")

df["script"] = df.code.map(lambda x:languages[x])
df

set swa to Latn script
set orm to Latn script
set mlg to Latn script
set ful to Latn script
set aka to Latn script
set lin to Latn script


,Unnamed: 0,language,n_speakers,code,script
0,0,Swahili,80,swa,swa_Latn
1,1,Hausa,70,hau,hau_Latn
2,2,Amharic,57,amh,amh_Ethi
3,3,Yoruba,45,yor,yor_Latn
4,4,Oromo,40,orm,orm_Latn
5,5,Igbo,40,ibo,ibo_Latn
6,6,Zulu,27,zul,zul_Latn
7,7,Malagasy,25,mlg,mlg_Latn
8,8,Fula,25,ful,ful_Latn
9,9,Somali,22,som,som_Latn


In [13]:
df.to_csv("data/languages.csv")
